# Get data from the *_config.log and *_stats.txt files

In [21]:
# Libraries for load data from log and stats files
import os
import pandas as pd
import numpy as np

In [22]:
NUM_FILE_TOTAL_LAKAGE = 15; INIT_COLUMN_TOTAL_LAKAGE = 19; END_COLUMN_TOTAL_LAKAGE = 25
NUM_FILE_RUN_DYNAMIC = 19; INIT_COLUMN_RUN_DYNAMIC = 21; END_COLUMN_RUN_DYNAMIC = 27
NUM_FILE_CPI = 17; INIT_COLUMN_CPI = 46; END_COLUMN_CPI = 54

In [23]:
total_lakage = []
run_dynamic = []
cpi = []
label = []

In [24]:
SIM_NAME = 'mp3_dec'
dir = 'log_results/{}/'.format(SIM_NAME)
for filename in os.listdir(dir):
    if filename.endswith('.log'):
        path = os.path.join(dir, filename)
        try:
            with open(path, 'r') as f:
                lines = f.readlines()
                if NUM_FILE_TOTAL_LAKAGE < len(lines):
                    data = lines[NUM_FILE_TOTAL_LAKAGE-1]
                    data = data[INIT_COLUMN_TOTAL_LAKAGE-1:END_COLUMN_TOTAL_LAKAGE]
                    total_lakage.append(float(data))
                if NUM_FILE_RUN_DYNAMIC < len(lines):
                    data = lines[NUM_FILE_RUN_DYNAMIC-1]
                    data = data[INIT_COLUMN_RUN_DYNAMIC-1:END_COLUMN_RUN_DYNAMIC]
                    run_dynamic.append(float(data))
        except FileNotFoundError:
            print(f"El archivo {filename} no fue encontrado.")
        except IOError:
            print(f"No se pudo leer el archivo {filename}.")

dir = 'database/{}/'.format(SIM_NAME)
for filename in os.listdir(dir):
    if filename.endswith('.txt'):
        base = os.path.splitext(filename)[0]; base =base.replace('_stats', '')
        label.append(base)
        path = os.path.join(dir, filename)
        try:
            with open(path, 'r') as f:
                lines = f.readlines()
                if NUM_FILE_CPI < len(lines):
                    data = lines[NUM_FILE_CPI-1]
                    data = data[INIT_COLUMN_CPI-1:END_COLUMN_CPI]
                    cpi.append(float(data))
        except FileNotFoundError:
            print(f"El archivo {filename} no fue encontrado.")
        except IOError:
            print(f"No se pudo leer el archivo {filename}.")

In [25]:
df = pd.DataFrame(columns=['Total Lakage', 'Runtime Dynamic', 'CPI', 'Energy', 'EDP', 'label'])
df['Total Lakage'] = total_lakage
df['Runtime Dynamic'] = run_dynamic
df['CPI'] = cpi
df['Energy'] = (df['Total Lakage'] + df['Runtime Dynamic']) * df['CPI']
df['EDP'] = df['Energy'] * df['CPI']
df['label'] = label
df.head()
df.to_csv('metrics_{}.csv'.format(SIM_NAME), index=False)